In [ ]:
import json
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import numpy as np
import torch

LANDMARKS_PATH = 'data.json'
ANNOT_PATH = 'SLOVO_DATAFRAME.tsv'

with open(LANDMARKS_PATH) as f:
    data = json.load(f)

annot = pd.read_table(ANNOT_PATH)

In [ ]:
#!pip install -q mediapipe

In [ ]:
# labels for training
labels = ['Привет!', 'Пока', 'Я', 'тебя', 'любить']
label_encoding = {label:i for i, label in enumerate(labels)}
inv_lbl_encoding = {value:key for key, value in label_encoding.items()}
label_encoding

In [ ]:
from data.data_utils import LandmarksDataset

# all filenames for chosen labels
filenames = [annot[annot['text']==lbl]['attachment_id'].values for lbl in labels]
# flatten the list of lists
filenames = [item for sublist in filenames for item in sublist]
# landmarks dict
landmarks = {filename:data[filename] for filename in filenames}

In [ ]:
labels_of_files = [label_encoding[annot[annot['attachment_id'] == filename].iloc[0]['text']] for filename in filenames]
train_ind, val_ind = train_test_split(np.arange(len(filenames)), 
                                      test_size=0.2, 
                                      stratify=labels_of_files)
# num frames in each video
seq_len = 32
train_dataset = LandmarksDataset(landmarks, [filenames[i] for i in train_ind],
                           annot, label_encoding, seq_len, mode='train')
val_dataset = LandmarksDataset(landmarks, [filenames[i] for i in val_ind],
                           annot, label_encoding, seq_len, mode='test')

In [ ]:
from sklearn.model_selection import train_test_split

train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True,
                                               batch_size=16, num_workers=2)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=16,
                                             num_workers=2)

Reading the model

In [ ]:
import torch.nn as nn
import torch.optim as optim
from model.LSTM_model import LSTM_model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_epochs = 3000

# Define the LSTM parameters
input_size = 63
hidden_size = 126
num_classes = len(labels)

# Instantiate the LSTM model
model = LSTM_model(input_size, hidden_size, num_classes).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

Initializing the wandb logger

In [ ]:
import wandb

wandb.login()

In [ ]:
run = wandb.init(project='rsl',
                group='train',
                entity = 'datasatanists',
                job_type='new')

Building train/test loop

In [ ]:
from utils.train_test_loop import train, test

max_acc = 0
best_model = model

for epoch in range(1, num_epochs+1):
    train(model, optimizer, train_dataloader, device, epoch, wandb, criterion)
    test_acc, model = test(model, val_dataloader, device, epoch, wandb, criterion)
    if test_acc > max_acc:
        best_model = model
        max_acc = test_acc
wandb.finish()

In [ ]:
print('best acc', max_acc)
torch.save(best_model, 'model.pt')